In [ ]:
%pip install tensorflow opencv-python-headless mtcnn

In [2]:
import cv2
import numpy as np
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from mtcnn.mtcnn import MTCNN
from cv2 import dnn
import matplotlib.pyplot as plt


In [3]:
# Load the model from the .h5 file
#model_path = 'faces_trained_model.h5'  # Make sure to provide the correct path
model_path = 'Deepfake2d.h5'
model = load_model(model_path)


In [4]:
# Load face detection model (MTCNN and SSD options)
def load_face_detection_model(method, model_paths=None):
    if method == 'MTCNN':
        return MTCNN()
    elif method == 'SSD':
        net = dnn.readNetFromCaffe(model_paths['prototxt'], model_paths['caffemodel'])
        return net
    else:
        print("Unsupported model for face detection.")
        return None

# Extract frames from video (randomly select a few)
def extract_frames_rand(video_path, num_frames_to_select):
    capture = cv2.VideoCapture(video_path)
    total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

    selected_frame_indices = random.sample(range(total_frames), num_frames_to_select)
    selected_frames = []

    for frame_idx in selected_frame_indices:
        capture.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = capture.read()
        if ret:
            selected_frames.append(frame)

    capture.release()
    return selected_frames

# Extract face from frame using either MTCNN or SSD
def extract_face(frame, method='MTCNN', margin=0):
    detector = None
    net = None
    if method == 'MTCNN':
        detector = load_face_detection_model(method)
        faces = detector.detect_faces(frame)
        if faces:
            x, y, w, h = faces[0]['box']
            x -= margin
            y -= margin
            w += 2 * margin
            h += 2 * margin
            x, y, w, h = max(0, x), max(0, y), w, h
            face = frame[y:y + h, x:x + w]
            return cv2.resize(face, (224, 224))
    elif method == 'SSD':
        model_paths = {'prototxt': 'deploy.prototxt.txt', 'caffemodel': 'res10_300x300_ssd_iter_140000.caffemodel'}
        net = load_face_detection_model(method, model_paths)
        (h, w) = frame.shape[:2]
        blob = dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                face = frame[startY:endY, startX:endX]
                return cv2.resize(face, (224, 224))
    return None


In [5]:
# Predict deepfake or real from extracted frames
def predict_from_frames(frames, face_detection_method='MTCNN'):
    faces = [extract_face(frame, method=face_detection_method) for frame in frames]
    faces = [face for face in faces if face is not None]

    if not faces:
        return [], []

    faces = np.array(faces)
    faces = preprocess_input(faces)

    # Perform prediction
    predictions = model.predict(faces)
    return predictions, predictions


In [6]:
# Simulate deepfake detection from a video
def predict_from_video(video_path, num_frames=3, face_detection_method='MTCNN'):
    frames = extract_frames_rand(video_path, num_frames)
    predictions, probs = predict_from_frames(frames, face_detection_method)

    if len(predictions) == 0:
        return {"error": "No Faces Detected"}, []

    return format_predictions(predictions, probs)

# Format predictions to be more readable
def format_predictions(predictions, probs):
    formatted_predictions = []
    for prediction, prob in zip(predictions, probs):
        label = "Fake" if prediction > 0.5 else "Real"
        probability = prob if label == "Fake" else 1 - prob
        formatted_predictions.append({"prediction": label, "probability": float(probability)})

    return formatted_predictions


In [ ]:
# Example video input path
video_path = 'awnfpubqmo.mp4'  # Provide a path to your video file

# Run deepfake detection on the video
predictions = predict_from_video(video_path, num_frames=3, face_detection_method='MTCNN')

# Print the results
print(predictions)


In [ ]:
# Show one of the extracted frames
def show_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

frames = extract_frames_rand(video_path, 1)
show_image(frames[0])


In [8]:
# Function to predict deepfake from an image
def predict_from_image(image_path, face_detection=True, face_detection_method='MTCNN'):
    # Load the image
    frame = cv2.imread(image_path)
    if frame is None:
        return {"error": "Image not found or unreadable"}, []

    # Perform face detection if required
    if face_detection:
        face = extract_face(frame, method=face_detection_method)
        if face is None:
            return {"error": "No Face Detected"}, []
        face = np.expand_dims(face, axis=0)
    else:
        # Resize and preprocess the image without face detection
        face = cv2.resize(frame, (224, 224))
        face = np.expand_dims(face, axis=0)

    # Preprocess the image (face or entire image)
    face = preprocess_input(face)

    # Perform prediction
    predictions = model.predict(face)

    # Format predictions
    return format_predictions(predictions, predictions)

# Example image input path
image_path = 'img.jpg'  # Provide a path to your image file

# Run deepfake detection on the image
predictions = predict_from_image(image_path, face_detection=True, face_detection_method='MTCNN')

# Print the results
print(predictions)


1/1 [==============================] - 0s 52ms/step
[{'prediction': 'Fake', 'probability': 0.7104664444923401}]


C:\Users\strea\AppData\Local\Temp\ipykernel_20536\2672792278.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  formatted_predictions.append({"prediction": label, "probability": float(probability)})
